# KNU 한국어 감성 사전을 이용하여 긍정 / 부정 label 생성하기

## 라이브러리 추가

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split

In [2]:
# 경고창 무시
import warnings
warnings.filterwarnings(action='ignore')

## 데이터 불러오기 및 전처리

In [3]:
review = pd.read_table('../data_list/ratings_total.txt', names=['review_score', 'review_content'])
review = review[:8000]
review[:3]

,review_score,review_content
0,5,배송 빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주 좋아요 바지 정말 좋아서2개 더 구매했어요 이 가격에 대박입니다. 바느질이 조...


In [4]:
review['review_content'] = review['review_content'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', x))
review['review_content'][:5]

0                                             배송 빠르고 굿
1                        택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2    아주 좋아요 바지 정말 좋아서개 더 구매했어요 이 가격에 대박입니다 바느질이 조금 ...
3    선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전화...
4                    민트 색상 예뻐요 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
Name: review_content, dtype: object

In [5]:
review.drop_duplicates(subset=['review_content'], inplace=True)

In [6]:
review['review_content'][:3]

0                                             배송 빠르고 굿
1                        택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2    아주 좋아요 바지 정말 좋아서개 더 구매했어요 이 가격에 대박입니다 바느질이 조금 ...
Name: review_content, dtype: object

## 사전 불러오기

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
positive = pd.read_csv('../data_list/positive_dictionary.csv', encoding='utf-8')
positive_dict = positive['word'].values
positive_dict.tolist()
print(positive_dict)

['가능성이 늘어나다' '가능성이 있다고' '가능하다' ... '상위권' '안정감' '적극적이다']


In [9]:
positive_word = ['빠르고', '굿', '더', '좋아', '정말', '대박', '빨리', '예뻐요',
                 '예쁘', '실용', '딱', '귀엽', '넘', '저렴', '좋고', '좋다', '좋아',
                 '편한', '편함', '편하다', '재구매', '구매', '가성비', '다만', '젤',
                 '빠른', '배송', '빠', '여유', '마음', '곱네요', '이쁨', '최고', '추가',
                 '좋은', '감사', '신용', '귀엽고', '생각보다', '저렴한', '모두', '맘에', 
                 '들다', '맘에는', '좋아요', 'ㅎㅎ']

In [10]:
positive_dict = np.append(positive_dict, positive_word)
positive_dict

array(['가능성이 늘어나다', '가능성이 있다고', '가능하다', ..., '맘에는', '좋아요', 'ㅎㅎ'],
      dtype=object)

In [11]:
negative = pd.read_csv('../data_list/negative_dictionary.csv', encoding='utf-8')
negative_dict = negative['word'].values
negative_dict.tolist()
print(negative_dict)

['(;_;)' '(^_^;' '(-_-)' ... '내팽개치다' '횡령' '불안증']


In [12]:
negative_word = ['엉망', '없이', '가고', '조금', '비추', '불편', '별로',
                 '안', '때', '작은', '빨', '비싼', '별루',
                 '몰랐', '이상', '오랫', '않아요', '불친절', '불쾌', '최악', '떨어짐',
                 '빠지', '없', '높아', '근데', '따로', '번졌', '않',
                 '깔끄', '의심', '폭망', '떡', '걱정']

In [13]:
negative_dict = np.append(negative_dict, negative_word)
negative_dict

array(['(;_;)', '(^_^;', '(-_-)', ..., '폭망', '떡', '걱정'], dtype=object)

## tfidf --> fit/transform

In [14]:
tfidf1 = TfidfVectorizer()
tfidf2 = TfidfVectorizer()

In [15]:
tfidf1.fit_transform(positive_dict).toarray()
tfidf2.fit_transform(negative_dict).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## label 생성

In [16]:
def pos_neg(df, column):
    df['label'] = np.nan
    count = len(df)
    for i in range(count):
        print('--------------------'+ str(i) +'--------------------')
        positive_words_count=tfidf1.transform([df[column][i]]).toarray().sum()
        negative_words_count=tfidf2.transform([df[column][i]]).toarray().sum()
        if positive_words_count - negative_words_count > 0:
            df.at[i, 'label'] = 1
        else:
            df.at[i, 'label'] = 0
        print(df.loc[i])

In [17]:
pos_neg(review, 'review_content')

--------------------0--------------------
review_score             5
review_content    배송 빠르고 굿
label                    1
Name: 0, dtype: object
--------------------1--------------------
review_score                                  2
review_content    택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
label                                         0
Name: 1, dtype: object
--------------------2--------------------
review_score                                                      5
review_content    아주 좋아요 바지 정말 좋아서개 더 구매했어요 이 가격에 대박입니다 바느질이 조금 ...
label                                                             1
Name: 2, dtype: object
--------------------3--------------------
review_score                                                      2
review_content    선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전화...
label                                                             0
Name: 3, dtype: object
--------------------4--------------------
review_score                                      5
review_conten

review_score                    2
review_content    바람이 나는지 잘 모르겠네요
label                           0
Name: 86, dtype: object
--------------------87--------------------
review_score                                                   2
review_content    크기도 생각보다 크고요심이 있어 쪄서 갈아 먹었는데도 안갈린건 입안에서 걸리더라구요
label                                                          1
Name: 87, dtype: object
--------------------88--------------------
review_score                  4
review_content    선물로보냈는데 잘받았되요
label                         0
Name: 88, dtype: object
--------------------89--------------------
review_score                     5
review_content    필수입니다 꼭 같이 구매하세요
label                            0
Name: 89, dtype: object
--------------------90--------------------
review_score                                            2
review_content    촉감 너무꺼끌하고 별로예요 시원한지도 잘모르겠고 반품귀찮아서 그냥씁니다
label                                                   0
Name: 90, dtype: object
--------------------91--------------

review_score                              2
review_content    목뿌러진게왔네요  검수잘해달라고 부탁드렸는데ㅠ
label                                     0
Name: 175, dtype: object
--------------------176--------------------
review_score                         4
review_content    재구매 잘먹고있습니다 아침식사로좋아요
label                                1
Name: 176, dtype: object
--------------------177--------------------
review_score                2
review_content    비개 가 너무 많아요
label                       0
Name: 177, dtype: object
--------------------178--------------------
review_score                                   5
review_content    입구가 조금 좁은거 말곤 재질이나 배송 모두 만족합니다
label                                          1
Name: 178, dtype: object
--------------------179--------------------
review_score                        5
review_content    재구매 좋아요 잘받았습니다 잘쓸께요
label                               1
Name: 179, dtype: object
--------------------180--------------------
review_score                                        1
r

review_score               2
review_content    배송이 넘 느리네요
label                      0
Name: 268, dtype: object
--------------------269--------------------
review_score                            1
review_content    앙고라 털이 엄청빠져요 옷에 다 붙어나와요
label                                   0
Name: 269, dtype: object
--------------------270--------------------
review_score                 5
review_content    예쁘고 깔끔 아부조아요
label                        1
Name: 270, dtype: object
--------------------271--------------------
review_score                     4
review_content    가서사는것보단 저렴한듯 좋아요
label                            1
Name: 271, dtype: object
--------------------272--------------------
review_score                   2
review_content    재구매 항상쓰던제품 입니다
label                          1
Name: 272, dtype: object
--------------------273--------------------
review_score                                       1
review_content    너무 써요 ㅠㅠㅠ 왠만해서 잘먹는데 표정이 막 일그러져요 ㅠㅠ
label                                  

review_score                    5
review_content    베송도 빠르고 상품도 좋아요
label                           1
Name: 366, dtype: object
--------------------367--------------------
review_score                        5
review_content    검은색 예쁨 굿굿신랑이 맘에 들어함
label                               1
Name: 367, dtype: object
--------------------368--------------------
review_score                               1
review_content    불량으로인한 판매자 마인드 불만족 사과도없었어요
label                                      0
Name: 368, dtype: object
--------------------369--------------------
review_score                                           5
review_content    항상 주문하는 제품으로 사이즈 좋고 무엇보다 손님들이 너무 맛있데요 
label                                                  1
Name: 369, dtype: object
--------------------370--------------------
review_score                                                      2
review_content    별루 사실 아이보리 사고싶엇는데 맘에 드는게 없어서 밝은쪽 쓰려고 투톤으로 고름 근...
label                                                             

...

review_score                                                 4
review_content    색깔을 세개선택하니 리뷰도 세개 따로 적어야 하네요 색깔 예쁘고 튼튼하니 좋아요
label                                                        1
Name: 7534, dtype: object
--------------------7535--------------------
review_score                        2
review_content    손잡이 부분이 흐물흐물해서 불편해요
label                               0
Name: 7535, dtype: object
--------------------7536--------------------
review_score               5
review_content    실내걷기에 짱이에요
label                      0
Name: 7536, dtype: object
--------------------7537--------------------
review_score                                                      1
review_content    리본 달려있는 디자인이 마음에 들어 세개 시켰고 오자마자 세탁망에 울코스로 약하게 ...
label                                                             0
Name: 7537, dtype: object
--------------------7538--------------------
review_score                            5
review_content    생각보다 튼튼하고 좋네요 무료배송도 좋아요
label                                 

review_score                         2
review_content    생각보다 작아요 머리 고정이 잘안되요
label                                1
Name: 7632, dtype: object
--------------------7633--------------------
review_score                                                      2
review_content    지독한 냄새 안빠져요 냄새 넘 심하고 며칠이 지나도 안빠지네요 결국 사용 못하고 에...
label                                                             0
Name: 7633, dtype: object
--------------------7634--------------------
review_score                                                      5
review_content    하이체어 삐그아이가 앉으면 금새내려달라그 울고불고ㅠㅠ해서 혹시나하는맘으로 사봤습니다...
label                                                             0
Name: 7634, dtype: object
--------------------7635--------------------
review_score                  5
review_content    좋아요 배송 잘되요 만족
label                         1
Name: 7635, dtype: object
--------------------7636--------------------
review_score                     4
review_content    잡기도 편하고 사용하기 좋아요
label                    

review_score                                                      1
review_content    상품의 질이 아주 나쁩니다 고구마가 젖어있는 것들도 있고 사이즈도 아주 적어서 내가...
label                                                             0
Name: 7729, dtype: object
--------------------7730--------------------
review_score                                  1
review_content    다른데에비해 푸석푸석 쫀득하지않네요 맛술넣어서그런가요
label                                         0
Name: 7730, dtype: object
--------------------7731--------------------
review_score                                        2
review_content    차망 주전자인데 차망 안주셔서 반품할까하다가 귀찮아서 그냥씁니다
label                                               0
Name: 7731, dtype: object
--------------------7732--------------------
review_score                                                      1
review_content    제품이 다 녹아서 완전 흐물흐물해요 재 냉동 하면 완전 떡될것같아서 바로 다 써야하네요ㅠ
label                                                             0
Name: 7732, dtype: object
--------------------7733--------------------
re

Name: 7826, dtype: object
--------------------7827--------------------
review_score                                                      2
review_content    양이 좀 적은편같고 아이스팩 완전 다 녹아서 그런지 새우랑야채도 흐물하고 상태가 좀...
label                                                             0
Name: 7827, dtype: object
--------------------7828--------------------
review_score             1
review_content    싼값하네요얇아요
label                    0
Name: 7828, dtype: object
--------------------7829--------------------
review_score                       2
review_content    보기보다 제퓸품질이 의심스럽습니다
label                              0
Name: 7829, dtype: object
--------------------7830--------------------
review_score                            5
review_content    소고기 생각나면 무조건 여기로 주문할랍니다
label                                   0
Name: 7830, dtype: object
--------------------7831--------------------
review_score                   2
review_content    가격만큼이네요 리뷰에 낚임
label                          0
Name: 7831, dtype: objec

review_score                                                5
review_content    공간이 확 넓어서 너무 좋아요 감사합니다 재구매해야할 경우가 생기겠어요 호호호
label                                                       1
Name: 7917, dtype: object
--------------------7918--------------------
review_score                   5
review_content    배송이 빨라서 참 좋았어요
label                          0
Name: 7918, dtype: object
--------------------7919--------------------
review_score             5
review_content    튼튼하고 이쁘요
label                    0
Name: 7919, dtype: object
--------------------7920--------------------
review_score                           5
review_content    무게가 살짝있으니깐 더욱더 안전한것같아요
label                                  1
Name: 7920, dtype: object
--------------------7921--------------------
review_score                    5
review_content    재구매 최고입니다 번창하세요
label                           1
Name: 7921, dtype: object
--------------------7922--------------------
review_score                 5
review_content    제품상태좋

In [18]:
review[:3]

,review_score,review_content,label
0,5,배송 빠르고 굿,1.0
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0.0
2,5,아주 좋아요 바지 정말 좋아서개 더 구매했어요 이 가격에 대박입니다 바느질이 조금 ...,1.0


In [19]:
review.isna().sum()

review_score      0
review_content    0
label             0
dtype: int64

## train_test_split --> fit / transform

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
            review['review_content'],
            review['label'],
            test_size=0.2,
            random_state=42
)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [22]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords = sum(stopwords, [])
stopwords[:10]

['휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해']

In [23]:
tfidf = TfidfVectorizer(lowercase = True, stop_words=stopwords)

In [24]:
tfidf.fit(X_train)

TfidfVectorizer(stop_words=['휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희',
                            '따라', '의해', '을', '를', '에', '의', '가', '으로', '로',
                            '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면',
                            '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', ...])

In [25]:
X_train =  tfidf.transform(X_train)

In [26]:
X_test= tfidf.transform(X_test)

In [27]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(6400, 25087) (6400,) (1600, 25087) (1600,)


In [28]:
tfidf.inverse_transform(X_train[:1])

[array(['좋아요', '쓰던거라', '걱정없네요'], dtype='<U79')]

In [29]:
tfidf.inverse_transform(X_test[:1])

[array(['적고', '소음도', '무엇보다'], dtype='<U79')]

## 정확도 측정

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [31]:
SA_lr = LogisticRegression(random_state = 0)

In [32]:
SA_lr.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [33]:
test_predict = SA_lr.predict(X_test)

In [34]:
print('긍정/부정 분석 정확도 : ', round(accuracy_score(y_test, test_predict), 5))

긍정/부정 분석 정확도 :  0.865


## 1개의 review에 대해 긍정 / 부정 판단

In [35]:
nr = review['review_content'][6]

In [36]:
nr = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", nr)
nr

'주문을 월 에 시켰는데 월 일에 배송이 왔네요 ㅎㅎㅎ 여기 회사 측과는 전화도 안되고 아무런 연락을 받을수가 없으니 답답하신 분들은 다른곳에서 사시는거 추천드립니다'

In [37]:
nr_tfidf = tfidf.transform([nr])
nr_tfidf

<1x25087 sparse matrix of type '<class 'numpy.float64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [38]:
nr_predict = SA_lr.predict(nr_tfidf)
nr_predict, nr_predict[0]

(array([0.]), 0.0)

In [39]:
if(nr_predict[0] == 1):
    print(nr, '==> 긍정적인 후기입니다')
else:
    print(nr, '==> 부정적인 후기입니다')

주문을 월 에 시켰는데 월 일에 배송이 왔네요 ㅎㅎㅎ 여기 회사 측과는 전화도 안되고 아무런 연락을 받을수가 없으니 답답하신 분들은 다른곳에서 사시는거 추천드립니다 ==> 부정적인 후기입니다


## DB에 넣을 데이터 label 붙이기

In [40]:
review2 = pd.read_csv('../data_list/review_202205161607.csv', encoding='utf-8')
review2

,review_idx,product_idx,user_id,review_content,review_img,review_score,review_rgstdate
0,1,1,admin,색감이 곱네요,good!.png,4.5,2022-05-13 15:10:51
1,3,1,aaaaaa1234,색이 오묘한게 정말 예뻐요 여름에 진짜 잘 입을 것 같아용 케키케,good!.png,5.0,2022-05-13 15:20:53
2,4,1,use06048,우선 옷은 되게 얇아요! 그리고 제가 키가 큰 편이라 너무 짧고 작을까봐 걱정했는데...,good!.png,4.5,2022-05-13 15:20:53
3,5,1,qwer,색감이 너무 마음에 들어요! 사이즈도 괜찮고 디자인도 예뻐요~,good!.png,4.0,2022-05-13 15:20:53
4,6,1,sdf,받고 나서 깜짝 놀랐네요 벽돌색이라고 하고 파세요 상품 사진에 나온 것보다 한참 어...,BAD.png,0.5,2022-05-13 15:20:53
5,7,1,use06048,예뻐요 짱짱 그냥 고민 없이 사세요 늦게 산 걸 후회하는 중..,well...png,3.5,2022-05-13 15:20:53
6,8,1,aaaaaa1234,배송은 빠르고 이런 스타일 좋아하는 사람은 좋아하겠고 레이어드 해서 입힐 생각,well...png,4.0,2022-05-13 15:20:53
7,9,1,qwer,생각보다 더 얇고 작아요 ㅎㅎ 편하게 입기 좋을거 같습니다,good!.png,3.0,2022-05-13 15:20:53
8,10,1,use06048,키 골격 큰편이어서 작을까봐 걱정했는데 생각보다 괜찮네영 여름에 롱스커트랑 닥마에 ...,well...png,4.5,2022-05-13 15:20:53
9,11,1,sdf,옷만 보면 이쁨 바지도 청이나 어두운 계열 잘 어울릴 거 같아여,good!.png,3.5,2022-05-13 15:20:53


In [41]:
def labelmaker(df, column):
    df['review_label'] = np.nan
    count = len(df)
    for i in range(count):
        print('----------------------'+str(i)+'-----------------------')
        content = df[column][i]
        print(content)
        content = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", content)
        content_tfidf = tfidf.transform([content])
        predict = SA_lr.predict(content_tfidf)
        if(predict[0] == 1):
            df['review_label'] = df['review_label'].astype(str)
            df.at[i, 'review_label'] = '1F60A'
        else:
            df['review_label'] = df['review_label'].astype(str)
            df.at[i, 'review_label'] = '1F621'
        print(df['review_label'][i])

In [42]:
labelmaker(review2, 'review_content')

----------------------0-----------------------
색감이 곱네요
1F621
----------------------1-----------------------
색이 오묘한게 정말 예뻐요 여름에 진짜 잘 입을 것 같아용 케키케
1F60A
----------------------2-----------------------
우선 옷은 되게 얇아요! 그리고 제가 키가 큰 편이라 너무 짧고 작을까봐 걱정했는데 입고다닐만 합니다! 다만 하이웨스트 바지와 함께 입어야 할 듯!
1F621
----------------------3-----------------------
색감이 너무 마음에 들어요! 사이즈도 괜찮고 디자인도 예뻐요~
1F60A
----------------------4-----------------------
받고 나서 깜짝 놀랐네요 벽돌색이라고 하고 파세요 상품 사진에 나온 것보다 한참 어두운 갈색이고 칙칙해요 디자인은 예쁜데 색 때문에 엄청 못생겨보여요 반품하려다가 그냥 입습니다 색 어두운 거 좋아하시는 분들만 사세요 사진같은 핑크색 생각하고 샀는데 실물보고 낚인기분드네요
1F621
----------------------5-----------------------
예뻐요 짱짱 그냥 고민 없이 사세요 늦게 산 걸 후회하는 중..
1F621
----------------------6-----------------------
배송은 빠르고 이런 스타일 좋아하는 사람은 좋아하겠고 레이어드 해서 입힐 생각
1F60A
----------------------7-----------------------
생각보다 더 얇고 작아요 ㅎㅎ 편하게 입기 좋을거 같습니다
1F60A
----------------------8-----------------------
키 골격 큰편이어서 작을까봐 걱정했는데 생각보다 괜찮네영 여름에 롱스커트랑 닥마에 코디하면 이쁠거 같아용
1F60A
----------------------9-------------

In [43]:
review2

,review_idx,product_idx,user_id,review_content,review_img,review_score,review_rgstdate,review_label
0,1,1,admin,색감이 곱네요,good!.png,4.5,2022-05-13 15:10:51,1F621
1,3,1,aaaaaa1234,색이 오묘한게 정말 예뻐요 여름에 진짜 잘 입을 것 같아용 케키케,good!.png,5.0,2022-05-13 15:20:53,1F60A
2,4,1,use06048,우선 옷은 되게 얇아요! 그리고 제가 키가 큰 편이라 너무 짧고 작을까봐 걱정했는데...,good!.png,4.5,2022-05-13 15:20:53,1F621
3,5,1,qwer,색감이 너무 마음에 들어요! 사이즈도 괜찮고 디자인도 예뻐요~,good!.png,4.0,2022-05-13 15:20:53,1F60A
4,6,1,sdf,받고 나서 깜짝 놀랐네요 벽돌색이라고 하고 파세요 상품 사진에 나온 것보다 한참 어...,BAD.png,0.5,2022-05-13 15:20:53,1F621
5,7,1,use06048,예뻐요 짱짱 그냥 고민 없이 사세요 늦게 산 걸 후회하는 중..,well...png,3.5,2022-05-13 15:20:53,1F621
6,8,1,aaaaaa1234,배송은 빠르고 이런 스타일 좋아하는 사람은 좋아하겠고 레이어드 해서 입힐 생각,well...png,4.0,2022-05-13 15:20:53,1F60A
7,9,1,qwer,생각보다 더 얇고 작아요 ㅎㅎ 편하게 입기 좋을거 같습니다,good!.png,3.0,2022-05-13 15:20:53,1F60A
8,10,1,use06048,키 골격 큰편이어서 작을까봐 걱정했는데 생각보다 괜찮네영 여름에 롱스커트랑 닥마에 ...,well...png,4.5,2022-05-13 15:20:53,1F60A
9,11,1,sdf,옷만 보면 이쁨 바지도 청이나 어두운 계열 잘 어울릴 거 같아여,good!.png,3.5,2022-05-13 15:20:53,1F621


In [44]:
review2.to_csv('../data_list/review_table2.csv', header=True, index=False)